In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct, avg,stddev,max,min
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [2]:
import os
os.environ['JAVA_HOME'] = 'D:\\Java'  # 这里的路径为java的bin目录所在路径

In [3]:
spark = SparkSession.builder.appName('hw2').getOrCreate()

In [4]:
customSchema = StructType([
  StructField("Year", IntegerType(), True),
  StructField("Month", IntegerType(), True),
  StructField("DayofMonth", IntegerType(), True),
  StructField("DayOfWeek", IntegerType(), True),
  StructField("DepTime", IntegerType(), True),
  StructField("CRSDepTime", IntegerType(), True),
  StructField("ArrTime", IntegerType(), True),
  StructField("CRSArrTime", IntegerType(), True),
  StructField("UniqueCarrier", StringType(), True),
  StructField("FlightNum", IntegerType(), True),
  StructField("TailNum", StringType(), True),
  StructField("ActualElapsedTime", IntegerType(), True),
  StructField("CRSElapsedTime", IntegerType(), True),
  StructField("AirTime", IntegerType(), True),
  StructField("ArrDelay", IntegerType(), True),
  StructField("DepDelay", IntegerType(), True),
  StructField("Origin", StringType(), True),
  StructField("Dest", StringType(), True),
  StructField("Distance", IntegerType(), True),
  StructField("TaxiIn", IntegerType(), True),
  StructField("TaxiOut", IntegerType(), True),
  StructField("Cancelled", IntegerType(), True),
  StructField("CancellationCode", StringType(), True),
  StructField("Diverted", IntegerType(), True),
  StructField("CarrierDelay", IntegerType(), True),
  StructField("WeatherDelay", IntegerType(), True),
  StructField("NASDelay", IntegerType(), True),
  StructField("SecurityDelay", IntegerType(), True),
  StructField("LateAircraftDelay", IntegerType(), True)]
)

In [5]:
df = spark.read.option("header","true").schema(customSchema).csv("*.csv")

In [6]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)

In [7]:
df.createOrReplaceTempView("schedule")

In [8]:
spark.sql("SELECT distinct(Year) FROM schedule order by Year").show()

+----+
|Year|
+----+
|2000|
|2001|
|2002|
|2003|
|2004|
|2005|
|2007|
|2008|
+----+



In [9]:
spark.sql("SELECT Month,max(ArrDelay) FROM schedule \
          where Year = 2008 group by Month order by Month").show()

+-----+-------------+
|Month|max(ArrDelay)|
+-----+-------------+
|    1|         1525|
|    2|         2461|
|    3|         1490|
|    4|         2453|
|    5|         1951|
|    6|         1707|
|    7|         1510|
|    8|         1359|
|    9|         1583|
|   10|         1392|
|   11|         1308|
|   12|         1655|
+-----+-------------+



In [10]:
spark.sql("SELECT Month,max(DepDelay) FROM schedule \
          where Year = 2008 group by Month order by Month").show()

+-----+-------------+
|Month|max(DepDelay)|
+-----+-------------+
|    1|         1355|
|    2|         2457|
|    3|         1521|
|    4|         2467|
|    5|         1952|
|    6|         1710|
|    7|         1518|
|    8|         1367|
|    9|         1552|
|   10|         1369|
|   11|         1286|
|   12|         1597|
+-----+-------------+



In [11]:
spark.sql("SELECT Month,max(ArrDelay),max(DepDelay) FROM schedule \
          where Year = 2008 group by Month order by Month").show()

+-----+-------------+-------------+
|Month|max(ArrDelay)|max(DepDelay)|
+-----+-------------+-------------+
|    1|         1525|         1355|
|    2|         2461|         2457|
|    3|         1490|         1521|
|    4|         2453|         2467|
|    5|         1951|         1952|
|    6|         1707|         1710|
|    7|         1510|         1518|
|    8|         1359|         1367|
|    9|         1583|         1552|
|   10|         1392|         1369|
|   11|         1308|         1286|
|   12|         1655|         1597|
+-----+-------------+-------------+



In [12]:
spark.sql("SELECT Year,count(WeatherDelay) FROM schedule \
          where Year>=2000 AND Year<=2005 group by Year order by Year").show()

+----+-------------------+
|Year|count(WeatherDelay)|
+----+-------------------+
|2000|                  0|
|2001|                  0|
|2002|                  0|
|2003|            3815798|
|2004|            7129270|
|2005|            7140596|
+----+-------------------+



In [27]:
spark.sql("SELECT Origin,count(*) FROM schedule \
          where Year = 2007 and\
          (CarrierDelay IS NOT NULL or WeatherDelay IS NOT NULL or \
          NASDelay IS NOT NULL or SecurityDelay IS NOT NULL or \
          LateAircraftDelay IS NOT NULL) \
          group by Origin order by 2 Desc LIMIT 5").show()

+------+--------+
|Origin|count(1)|
+------+--------+
|   ATL|  413851|
|   ORD|  375784|
|   DFW|  297345|
|   DEN|  240928|
|   LAX|  237597|
+------+--------+



In [26]:
spark.sql("SELECT Dest,count(*) FROM schedule \
          where Year = 2007 and\
          (CarrierDelay IS NOT NULL or WeatherDelay IS NOT NULL or \
          NASDelay IS NOT NULL or SecurityDelay IS NOT NULL or \
          LateAircraftDelay IS NOT NULL) \
          group by Dest order by 2 Desc LIMIT 5").show()

+----+--------+
|Dest|count(1)|
+----+--------+
| ATL|  413805|
| ORD|  375716|
| DFW|  297481|
| DEN|  240758|
| LAX|  237676|
+----+--------+



In [28]:
spark.sql("SELECT Origin,count(*) FROM schedule \
          where Year = 2007 and\
          DepDelay > 0 \
          group by Origin order by 2 Desc LIMIT 5").show()

+------+--------+
|Origin|count(1)|
+------+--------+
|   ATL|  206118|
|   ORD|  183984|
|   DFW|  135433|
|   DEN|  109839|
|   PHX|  105917|
+------+--------+



In [30]:
spark.sql("SELECT Dest,count(*) FROM schedule \
          where Year = 2007 and\
          ArrDelay > 0 \
          group by Dest order by 2 Desc LIMIT 5").show()

+----+--------+
|Dest|count(1)|
+----+--------+
| ATL|  186911|
| ORD|  177054|
| DFW|  134824|
| DEN|  110417|
| LAX|  109643|
+----+--------+



In [14]:
spark.sql("SELECT Origin,max(DepDelay) FROM schedule \
          where Year = 2007 group by Origin order by 2 Desc").show()

+------+-------------+
|Origin|max(DepDelay)|
+------+-------------+
|   PBI|         2601|
|   ALO|         1956|
|   HNL|         1831|
|   FWA|         1736|
|   FAI|         1689|
|   PIT|         1677|
|   MSN|         1657|
|   JFK|         1547|
|   IAH|         1529|
|   AUS|         1523|
|   ANC|         1487|
|   PDX|         1433|
|   SBN|         1429|
|   MEM|         1415|
|   GRR|         1408|
|   CLE|         1406|
|   FLL|         1372|
|   LAS|         1369|
|   KOA|         1360|
|   BUF|         1355|
+------+-------------+
only showing top 20 rows



In [15]:
spark.sql("SELECT Origin,max(ArrDelay),max(DepDelay) FROM schedule \
          where Year = 2007 group by Origin order by 2 Desc,3 Desc").show()

+------+-------------+-------------+
|Origin|max(ArrDelay)|max(DepDelay)|
+------+-------------+-------------+
|   PBI|         2598|         2601|
|   ALO|         1942|         1956|
|   HNL|         1848|         1831|
|   FWA|         1715|         1736|
|   FAI|         1665|         1689|
|   PIT|         1655|         1677|
|   MSN|         1652|         1657|
|   JFK|         1564|         1547|
|   AUS|         1534|         1523|
|   IAH|         1532|         1529|
|   ANC|         1469|         1487|
|   SBN|         1437|         1429|
|   CLE|         1426|         1406|
|   GRR|         1425|         1408|
|   PDX|         1418|         1433|
|   MEM|         1402|         1415|
|   BOI|         1387|         1345|
|   LAS|         1386|         1369|
|   FLL|         1379|         1372|
|   BUF|         1375|         1355|
+------+-------------+-------------+
only showing top 20 rows



In [17]:
spark.sql("SELECT Origin,max(ArrDelay),max(DepDelay) FROM schedule \
          where Year = 2007 group by Origin order by 2 Desc LIMIT 5").show()

+------+-------------+-------------+
|Origin|max(ArrDelay)|max(DepDelay)|
+------+-------------+-------------+
|   PBI|         2598|         2601|
|   ALO|         1942|         1956|
|   HNL|         1848|         1831|
|   FWA|         1715|         1736|
|   FAI|         1665|         1689|
+------+-------------+-------------+



In [18]:
spark.sql("SELECT Origin,max(ArrDelay),max(DepDelay) FROM schedule \
          where Year = 2007 group by Origin order by 3 Desc LIMIT 5").show()

+------+-------------+-------------+
|Origin|max(ArrDelay)|max(DepDelay)|
+------+-------------+-------------+
|   PBI|         2598|         2601|
|   ALO|         1942|         1956|
|   HNL|         1848|         1831|
|   FWA|         1715|         1736|
|   FAI|         1665|         1689|
+------+-------------+-------------+

